In [ ]:
### Uncomment the following lines to install the required packages###
# # For Azure AI Projects client library from Azure AI Foundry SDK packages and services
# ! pip install azure-ai-projects 
# # For Microsoft Entra ID token authentication support 
# ! pip install azure-identity 

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType
from azure.identity import DefaultAzureCredential
import os

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='<Your Project connection string from Azure AI Foundry>',
)

In [ ]:
search_connection = project_client.connections.get(
    connection_name='<Your AI Search connection name>',
)

In [ ]:
ai_search = AzureAISearchTool(search_connection.id, index_name="<Your AI Search index name>")

In [ ]:
with project_client:

    agent = project_client.agents.create_agent(
        model="gpt-4",
        name="ai-search-agent",
        instructions="You are helpful agent",
        tools=ai_search.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    thread = project_client.agents.create_thread()
    print(f"Thread ID: {thread.id}")
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="How does earth appear at night from satelites?",
    )
    print(f"Message ID: {message.id}")
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    messages = project_client.agents.list_messages(thread_id=thread.id)

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")
